In [1]:
from pathlib import Path


In [2]:
import spacy

model = spacy.load("fr_dep_news_trf")

/home/lerner/anaconda3/envs/lightning/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import re

In [4]:
inclusive_root = Path('data/corpus-francais-inclusif/vfi/')

# Syntax (coordination) "toutes et tous"

In [47]:
def is_masc(token):
    gender = token.morph.get("Gender")
    return gender and gender[0] == "Masc" 

def keep_masc(a,b):
    if is_masc(a):
        return a
    return b

extra_spaces = re.compile(r'\s+')

SEP ="·"
sep_sub = rf"\1{SEP}\2"
extra_seps = re.compile(r'(\w)[\.•](\w)')

In [48]:
def preproc(text):
    # common preproc: strip extra spaces
    text = extra_spaces.sub(' ', text.strip())
    # keep a single separator : "·"
    text = extra_seps.sub(sep_sub, text)
    # hack : run twice in case of things like "tou.s.tes" where "u.s" matching will prevent "s.t" matching
    text = extra_seps.sub(sep_sub, text)
    
    return text

In [56]:
print("\t".join(("a","b")))

a	b


In [53]:
def excl(tokens):
    for sentence in tokens.sents:
        todo = []
        for token in sentence:
            if token.dep_=="ROOT" or token.pos_ in {"VERB","AUX"} or token.is_space:
                continue
            if token.lemma == token.head.lemma and token.text!=token.head.text:
                print(token,token.head,token.dep_,token.pos_,token.head.dep_,token.head.pos_,end=" ; ")   
                i = token.i-sentence.start
                j = token.head.i-sentence.start
                if j < i:
                    i, j = j, i
                todo.append((i, j, keep_masc(token.head,token)))
        if not todo:
            x_sentence = sub(sentence.text)
            if x_sentence != sentence.text:                
                datum=(sentence.text, x_sentence)
                print(datum)
                yield datum   
            continue
        pi, pj, ptoken = todo[0]
        x_sentence = [sentence[:pi].text_with_ws, ptoken.text_with_ws]
        for i, j, token in todo[1:]:
            x_sentence.append(sentence[pj+1:i].text_with_ws)
            x_sentence.append(token.text_with_ws)
            pi, pj, ptoken = i, j, token
        x_sentence.append(sentence[pj+1:].text_with_ws)
        x_sentence = "".join(x_sentence)
        x_sentence = sub(x_sentence)
        datum=(sentence.text, x_sentence)
        print(datum)
        yield datum       

In [54]:
data = []
for path in inclusive_root.glob('*.txt'):
    with open(path, 'rt') as file:
        text = file.read()
    text = preproc(text)
    print(text)
    with open(path.with_suffix(".tag"),'rt') as file:
        print(re.findall(r'<Inclusif.+>(.+)</Inclusif>',file.read()))
    #print(inclusif.findall(text))
    tokens = model(text)
    for datum in excl(tokens):
        data.append(datum)
    print("\n******************\n")
    #break

Je m'adresse à celles et ceux qui refusent que d'autres décident à leur place, qui veulent se faire entendre et prendre leurs affaires en main.
[]
ceux celles conj PRON obl:arg PRON ; ("Je m'adresse à celles et ceux qui refusent que d'autres décident à leur place, qui veulent se faire entendre et prendre leurs affaires en main.", "Je m'adresse à ceux qui refusent que d'autres décident à leur place, qui veulent se faire entendre et prendre leurs affaires en main.")

******************

Je serai la Présidente de la réconciliation. Je respecterai chacune et chacun d'entre vous. Je serai Présidente pour tous les Français.
[]

******************

Mes chers amis, celles et ceux qui sont à l'intérieur, celles et ceux qui sont au dehors, car je m'excuse auprès d'eux, le chapiteau était trop petit, mais j'espère que la sonorisation leur permet de nous entendre... mes chers amis, j'étais à Lille il y a quinze jours et j'avais dit que j'essaierais de revenir dans le Pas-de-Calais. Les Françaises 

# Regex (écriture inclusive) "tous.tes"

In [ ]:
# fem_suffix_and_sub = [
#     ("esse", "e"),
#     ("sse", ""),
#     ("e",""), 
# ]

In [38]:
fem_suffix = r"(esse|sse|e|euse|se|ienne|enne|nne|ne|ère|ere|re|trice|rice|ice)"
inclusif_before_sub = r"\2\3"
inclusif_before_expr = re.compile(rf"{fem_suffix}s?{SEP}(\w+)(s?)\b")
inclusif_after_sub = r"\1\3"
inclusif_after_expr = re.compile(rf"{SEP}?(\w+)?{SEP}{fem_suffix}(s?)\b")
#inclusif_sing = re.compile(rf"({seps}{fem_suffix})|(\({fem_suffix}\))\b")
#inclusif_plur = re.compile(rf"({seps}{fem_suffix}({seps}?s))|(\({fem_suffix}s\))|(\({fem_suffix}\)s)\b")

In [39]:
def sub(text):
    text = inclusif_before_expr.sub(inclusif_before_sub,text)
    text = inclusif_after_expr.sub(inclusif_after_sub,text)
    return text

In [64]:
text="""Il s'agit de donner les moyens à chacun·e de satisfaire ses besoins mais aussi de s'émanciper dans une société plus démocratique, où la solidarité prendra le pas sur les égoïsmes.

Fin de l'élevage industriel, maltraitant pour les animaux et les salarié·e·s. Objectif 100% bio, développement des circuits courts.

Moratoire sur les dettes des agriculteurs·trices. Fonds public pour l'achat de foncier afin d'installer de jeunes agriculteurs·trices en coopérative.

Indemnités d'élu·e plafonnées au salaire médian. Révocabilité des élu·e·s.

Nous voulons une société solidaire et sans frontière, qui accueille tou·te·s les migrant·e·s

Droit de vote pour les étranger·e·s résidant en France"""

In [51]:
fix_s = re.compile(r"ss\b") # fixes erratic usages like "agriculteurS.ices" vs "agriculteur.ices"

def sub(text):
    text = inclusif_plur.sub('s', text)
    text = inclusif_sing.sub('', text)
    text = fix_s.sub('s', text)
    return text